# **SGAN:**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import joblib

In [3]:
df = pd.read_csv('/content/drive/My Drive/dataset/final_train.csv', encoding='ISO-8859-1')
df.head(10)

,Patient_comment,Patient_Category,Doctor_Type,Sentiment
0,rememb feel,Emotional pain,Psychiatrist,0.00
1,carri heavi thing feel like break back,Hair falling out,Dermatologist,0.00
2,much pain move arm,Heart hurts,Cardiologist,0.20
3,son lip pierc swollen skin insid lip grey look...,Infected wound,Surgeon,-0.05
4,muscl lower back ach,Infected wound,Surgeon,0.00
5,muscl pain back muscl pain back leg,Foot ache,Rheumatologist,0.00
6,muscl pain left leg,Shoulder pain,Rheumatologist/Gastro-enterologist,0.00
7,cut finger play footbal appli pain relief crea...,Injury from sports,Rheumatologist,0.00
8,acn face problem derma like itch,Skin issue,Dermatologist,0.00
9,strang rash arm,Foot ache,Rheumatologist,0.00


In [4]:
!pip install transformers
!pip install torch torchvision torchaudio

In [4]:
label_encoder = LabelEncoder()
df['Doctor_Type'] = label_encoder.fit_transform(df['Doctor_Type'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['Patient_comment'], df['Doctor_Type'], test_size=0.1, random_state=42)


In [6]:
# Load the BioBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

# Tokenize the input text
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

In [7]:
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = MedicalDataset(train_encodings, y_train.to_numpy())
test_dataset = MedicalDataset(test_encodings, y_test.to_numpy())

In [8]:
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=len(label_encoder.classes_))


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import Trainer, TrainingArguments

# Define the logging directory
logging_dir = '/content/drive/My Drive/dataset/log_file(3)'

training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=25,              # total number of training epochs
    per_device_train_batch_size=8,    # batch size per device during training
    per_device_eval_batch_size=16,    # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir=logging_dir,          # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",      # evaluate at the end of each epoch
    save_strategy="epoch",            # save the model at the end of each epoch
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')


    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Add the metrics function here
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.005700,0.075587,0.985007,0.985407,0.985007,0.984962
2,0.001100,0.032866,0.995502,0.995632,0.995502,0.995477
3,0.000800,0.020724,0.997001,0.997038,0.997001,0.996990
4,0.079200,0.012685,0.998501,0.998517,0.998501,0.998492
5,0.000600,0.019899,0.997001,0.997049,0.997001,0.996960
6,0.000300,0.009451,0.998501,0.998526,0.998501,0.998494
7,0.000200,0.011227,0.998501,0.998526,0.998501,0.998494
8,0.000200,0.011294,0.998501,0.998517,0.998501,0.998492
9,0.000100,0.013961,0.998501,0.998526,0.998501,0.998494
10,0.000200,0.008030,0.998501,0.998526,0.998501,0.998494


TrainOutput(global_step=18750, training_loss=0.052299137093817505, metrics={'train_runtime': 1749.9306, 'train_samples_per_second': 85.632, 'train_steps_per_second': 10.715, 'total_flos': 1617305142133800.0, 'train_loss': 0.052299137093817505, 'epoch': 25.0})

In [13]:
save_dir = '/content/drive/My Drive/Dakshaab/saved_model'

In [14]:
from transformers import BertForSequenceClassification, BertTokenizer
import joblib

# Ensure the directory exists (this will create it if it doesn't exist)
import os
os.makedirs(save_dir, exist_ok=True)

# Save the model to the specified directory
trainer.save_model(save_dir)  # This saves the model, tokenizer, and configuration
tokenizer.save_pretrained(save_dir)  # Save the tokenizer

# Save the label encoder to the same directory
joblib.dump(label_encoder, os.path.join(save_dir, 'label_encoder.pkl'))

print(f"Model, tokenizer, and label encoder saved to {save_dir}")


Model, tokenizer, and label encoder saved to /content/drive/My Drive/Dakshaab/saved_model


In [ ]:

# # Load the saved model and tokenizer
# model = BertForSequenceClassification.from_pretrained('./saved_model')
# tokenizer = BertTokenizer.from_pretrained('./saved_model')
# label_encoder = joblib.load('./saved_model/label_encoder.pkl')

# # Sample input comments
# comments = [
#     "The doctor was very attentive and kind.",
#     "I am dissatisfied with the service."
# ]

# # Tokenize the input comments
# encodings = tokenizer(comments, truncation=True, padding=True, max_length=512, return_tensors='pt')

# # Make predictions
# model.eval()
# with torch.no_grad():
#     outputs = model(**encodings)
#     predictions = torch.argmax(outputs.logits, dim=-1)

# # Decode predictions back to labels
# predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())

# # Display predictions
# for comment, label in zip(comments, predicted_labels):
#     print(f"Comment: {comment}\nPredicted Doctor Type: {label}\n")


In [24]:


# Specify the directory where the model, tokenizer, and label encoder are saved
save_dir = '/content/drive/My Drive/Dakshaab/saved_model'

# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained(save_dir)
tokenizer = BertTokenizer.from_pretrained(save_dir)
label_encoder = joblib.load(f'{save_dir}/label_encoder.pkl')

# Sample input comments
comments = [
    "I just hit with a cricket bat but there is no blood coming from it.",
    "there is itching in between my leg fingers"
]

# Tokenize the input comments
encodings = tokenizer(comments, truncation=True, padding=True, max_length=512, return_tensors='pt')

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(**encodings)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Decode predictions back to labels
predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())

# Display predictions
for comment, label in zip(comments, predicted_labels):
    print(f"Comment: {comment}\nPredicted Doctor Type: {label}\n")


Comment: I just hit with a cricket bat but there is no blood coming from it.
Predicted Doctor Type: Rheumatologist

Comment: there is itching in between my leg fingers
Predicted Doctor Type: Dermatologist

